In [1]:
import numpy as np
%set_env GOOGLE_APPLICATION_CREDENTIALS=/Users/drake/.ssh/Comp119-882760228b77.json
import json
from google.cloud import storage
storage_client = storage.Client()
bucket = storage_client.get_bucket("comp119")
blob = storage.blob.Blob("twitter_creds.json", bucket)

creds_bytes = blob.download_as_string()
_creds = json.loads(str(creds_bytes, 'utf-8'))
import tweepy
auth = tweepy.OAuthHandler(_creds["consumer_key"], _creds["consumer_secret"])
auth.set_access_token(_creds["access_token"], _creds["access_token_secret"])
del _creds # delete _creds immediately!
API = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

env: GOOGLE_APPLICATION_CREDENTIALS=/Users/drake/.ssh/Comp119-882760228b77.json


In [2]:
def show_tweet(tweet):

    try:
        return ' '.join([tweet.created_at.strftime("%m/%d/%Y, %H:%M:%S"),tweet.user.screen_name,tweet.text])

    except:
        return '********** bad or no tweet'

In [3]:
nyt = API.get_user('@nytimes')
user_tweets = API.user_timeline('@realDonaldTrump')
for tweet in user_tweets[:3]:
    print(show_tweet(tweet))
# Some followers of your favorite user
user_friends = API.friends('@nytimes')
print ([friend.screen_name for friend in user_friends])
tweeters = [nyt.id_str] + [friend.id_str for friend in user_friends]
# tweeters

12/09/2019, 17:27:11 realDonaldTrump RT @realDonaldTrump: The Real Deal! https://t.co/jUXHPU85PQ
12/09/2019, 15:47:52 realDonaldTrump Read the Transcripts!
12/09/2019, 15:25:21 realDonaldTrump True! https://t.co/1seRjQ11ru
['uwsgeezer', 'davidenrich', 'marywalshnyt', 'collinskeith', 'johnnydiaz__', 'timesrunclub', 'cflav', 'aimee_ortiz', 'S_Evangelina', 'ShaneGoldmacher', 'kashhill', 'antontroian', 'kylebuchanan', 'NYTSports', 'ElianPeltier', 'eringriffith', 'FlitterOnFraud', 'gillianwong', 'Nataliekitro', 'caitlinroper']


In [4]:
followers_list = []
for page in tweepy.Cursor(API.followers_ids,screen_name = '@nytimes').pages():
    followers_list.extend(page)
    if len(followers_list) > 10:
        break
follow = followers_list
follow = np.array(map(str, follow))

In [5]:
import sys
class Listener(tweepy.streaming.StreamListener):
    def __init__(self, output_file=sys.stdout):
        super(Listener,self).__init__()
        self.output_file = output_file
    def on_status(self, tweet):
        print(show_tweet(tweet), file=self.output_file)
    def on_error(self, status_code):
        print(status_code)
        return False
listener = Listener()
stream = tweepy.Stream(auth=API.auth, listener=listener)
print ('Starting!!!!!!!!!!!!!!!!!!')
try:
    print('Start streaming.')
    stream.filter(follow=tweeters)
#stream.sample(languages=['en'])
except KeyboardInterrupt:
    print("Stopped.")
finally:
    stream.disconnect()
print('Done.')

Starting!!!!!!!!!!!!!!!!!!
Start streaming.
12/09/2019, 21:10:27 mkink RT @ShaneGoldmacher: Inbox: Buttigieg Campaign Announces Fundraisers Will Be Open and Names of People Raising Money Will Be Made Public
12/09/2019, 21:10:29 TymRhyms RT @nytimes: "You’re kind of trying to capture something spontaneous and ordinary, but you know you’re only going to use seconds of it." Wa…
Stopped.
Done.


In [6]:
with open('/Users/drake/Desktop/index.json', 'r') as f:
    d = json.loads(f.read())

In [ ]:
import sys
import socket
import tweepy
class Listener(tweepy.streaming.StreamListener):
    def __init__(self, c_socket, output_file=sys.stdout):
        super(Listener,self).__init__()
        self.output_file = open(output_file,'w')
        self.c_socket = c_socket
        self.tweet_list = []
        
    def on_status(self, tweet):
        print((str(tweet.id) + ' ' + tweet.text).replace('\n',' ') + '\n')
        
        self.c_socket.send(((str(tweet.id) + ' ' + tweet.text).replace('\n',' ') + '\n').encode("utf-8"))
        #self.c_socket.send(str(d["abandon"]).encode("utf-8"))
        
    def on_error(self, status_code):
        print(status_code)
        return False
    
    def __def__(self):
        self.output_file.close()


try:
    TCP_IP = "localhost"
    TCP_PORT = 10285
    conn = None
    s = socket.socket()
    s.bind((TCP_IP, TCP_PORT))
    s.listen(1)
    print("Waiting for TCP connection...")
    conn, addr = s.accept()
    print("Connected... Starting getting tweets.")

    listener = Listener(output_file='myoutput40.txt', c_socket=conn)
    stream = tweepy.Stream(auth=API.auth, listener=listener)
    print ('Starting!!!!!!!!!!!!!!!!!!')
    #stream.filter(follow=[nyt.id_str])
    print('Start streaming.')
    stream.filter(follow=tweeters)
#stream.sample(languages=['en'])
except KeyboardInterrupt:
    print("Stopped.")
finally:
    stream.disconnect()
    s.close()
    print('Done.')

Waiting for TCP connection...
Connected... Starting getting tweets.
Starting!!!!!!!!!!!!!!!!!!
Start streaming.
1204324714530902016 RT @nytimes: France and a group of Brazilian states plan to announce a partnership to preserve the Amazon rainforest, the group’s leader sa…

1204324742699868160 RT @nytimes: Manipulated Twitter accounts, doctored videos, dodgy websites and questions of foreign meddling. In just 6 weeks, the campaign…

1204324751469973504 @nytimes This site has no paywall  https://t.co/lp7ECIGwJD

1204324772303163397 RT @nytimes: A last-minute booking, a furtive cab ride and a spy in the window. How our correspondent found a crack in China’s surveillance…

1204324833359732736 RT @nytimes: In Opinion  Anu Partanen and Trevor Corson write, “The difference between our lives here and in the States has been tremendous…

1204324842100518913 RT @nytimes: "The consequences of ignoring the condition can be severe, but the health system puts up barriers to treatment," write Aaron C…

1204326015071162368 RT @nytimes: The biggest actors’ union is investigating “America’s Got Talent” and the leader of the group Time’s Up is taking aim at NBC f…

1204326022411411458 RT @nytimes: Away from the politics, Brexit has brought a new level of uncertainty for many of the people who go to work each day in Britai…

1204326066455793664 RT @eringriffith: The perfect coda to the Age of Unicorns:   A YC-backed scooter startup called Unicorn spent all its investor money on Fac…

1204326101645836288 @nytimes So he was using 'his' account, right?

1204326144826368000 RT @nytimes: Once a champion for human rights, Daw Aung San Suu Kyi will lead Myanmar’s defense against accusations of genocide on Tuesday…

1204326157052723200 @nytimes Time to leave.

1204326162404724736 @nytimes Please have a look on my profile for (Promotional purpose) https://t.co/IMLurZdK07

1204326180457000960 RT @eringriffith: The perfect coda to the Age of Unicorns:   A YC-backed scooter startup called Unicorn spe

1204327904257036289 RT @nytimes: Documents detailing the war in Afghanistan obtained by The Washington Post paint a stark picture of missteps and failures in t…

1204327952365699073 RT @nytimes: Europe morning briefing: Here's what you need to know to start your day https://t.co/4MPXJFOeA5

1204328017100718080 RT @nytimes: "The consequences of ignoring the condition can be severe, but the health system puts up barriers to treatment," write Aaron C…

1204328043289944064 RT @nytimes: Manipulated Twitter accounts, doctored videos, dodgy websites and questions of foreign meddling. In just 6 weeks, the campaign…

1204328100688994305 @nytimes And ALL of those can be attributed to the Tories!!

1204328121320853505 RT @nytimes: Manipulated Twitter accounts, doctored videos, dodgy websites and questions of foreign meddling. In just 6 weeks, the campaign…

1204328140169986048 RT @nytimes: Documents detailing the war in Afghanistan obtained by The Washington Post paint a stark picture of missteps